## Importing

In [ ]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=13c3c704b850a4dd6f0d32aa9eecaea612823d62c742a103d710afa47608134e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00


In [ ]:
pip install --upgrade peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.13.2
    Uninstalling peft-0.13.2:
      Successfully uninstalled peft-0.13.2


In [ ]:
import torch
from torch.utils.data import Dataset
import numpy as np
from rouge_score import rouge_scorer
from bert_score import score
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5Config
from safetensors.torch import load_file
import json
import os
from peft import PeftModel, PeftConfig
import nltk
import pandas as pd

In [ ]:
from transformers import logging

logging.set_verbosity_error()

In [ ]:
# Download NLTK data
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker_tab')
nltk.download('words')
nltk.download('punkt_tab')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker_tab is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Main Assessment

In [ ]:
def generate_qa(context, model, tokenizer):
    print("Context: ")
    print(context)
    inputs = tokenizer(context, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=500, num_return_sequences=1, num_beams=4)
    question_answer = tokenizer.decode(outputs[0], skip_special_tokens=False)
    print("Generated QA: ")
    print(question_answer)
    print("-" * 50)
    question_answer = question_answer.replace(tokenizer.pad_token, "").replace(tokenizer.eos_token, "")

    if tokenizer.sep_token in question_answer:
        question, answer = question_answer.split(tokenizer.sep_token, 1)
        generated_qa = f"{question} {answer}"
    else:
        generated_qa = question_answer

    return generated_qa

def compute_rouge_l(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_result = scorer.score(reference, candidate)
    precision = rouge_result['rougeL'].precision
    recall = rouge_result['rougeL'].recall
    f1_score = rouge_result['rougeL'].fmeasure
    return precision, recall, f1_score

def compute_bert_score(reference, candidate):
    precision, recall, f1 = score([candidate], [reference], lang="en", verbose=False)
    return precision.mean().item(), recall.mean().item(), f1.mean().item()


def evaluate_context_matching(contexts, model, tokenizer):
    rouge_metrics = {'precision': [], 'recall': [], 'f1': []}
    bert_metrics = {'precision': [], 'recall': [], 'f1': []}

    for context in contexts:
        generated_qa = generate_qa(context, model, tokenizer)

        rouge_precision, rouge_recall, rouge_f1 = compute_rouge_l(context, generated_qa)
        rouge_metrics['precision'].append(rouge_precision)
        rouge_metrics['recall'].append(rouge_recall)
        rouge_metrics['f1'].append(rouge_f1)

        bert_precision, bert_recall, bert_f1 = compute_bert_score(context, generated_qa)
        bert_metrics['precision'].append(bert_precision)
        bert_metrics['recall'].append(bert_recall)
        bert_metrics['f1'].append(bert_f1)

    summary = {
        'metrics': {metric: np.mean(values) for metric, values in bert_metrics.items()}
    }

    return summary

def preprocess_function_with_nltk(text):
    tokens = nltk.word_tokenize(text)

    pos_tags = nltk.pos_tag(tokens)
    pos_str = " ".join([f"{word}/{pos}" for word, pos in pos_tags])

    ner_tree = nltk.ne_chunk(pos_tags)
    ner_tags = " ".join([f"{' '.join(c[0] for c in subtree)}({subtree.label()})" if isinstance(subtree, nltk.Tree) else f"{subtree[0]}" for subtree in ner_tree])

    return ner_tags, pos_str

In [ ]:
# Load dataset using pandas
splits = {
    'train': 'data/train-00000-of-00001.parquet',
    'validation': 'data/validation-00000-of-00001.parquet',
    'test': 'data/test-00000-of-00001.parquet'
}
train_df = pd.read_parquet("hf://datasets/allenai/sciq/" + splits["train"])
validation_df = pd.read_parquet(
    "hf://datasets/allenai/sciq/" + splits["validation"])
test_df = pd.read_parquet("hf://datasets/allenai/sciq/" + splits["test"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
contexts = test_df["support"]

contexts.replace('', np.nan, inplace=True)
contexts.dropna(how='all', inplace=True)

contexts = list(contexts)

contexts = contexts[:10]

print(len(contexts))

10


In [ ]:
# contexts

In [ ]:
def evaluate_model_tag(contexts):
    finetuned_model_dir = "/content/finetuned_squad"

    finetuned_tokenizer = AutoTokenizer.from_pretrained(finetuned_model_dir)
    finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(finetuned_model_dir)

    finetuned_model = PeftModel.from_pretrained(finetuned_model, finetuned_model_dir)

    tagged_contexts_ner = []
    tagged_contexts_pos = []
    for context in contexts:
        ner_tags, pos_tags = preprocess_function_with_nltk(context)
        tagged_contexts_ner.append(ner_tags)
        tagged_contexts_pos.append(pos_tags)

    finetuned_metrics = evaluate_context_matching(contexts, finetuned_model, finetuned_tokenizer)

    print("Fine-tuned Model Metrics:", finetuned_metrics)

In [ ]:
evaluate_model_tag(contexts)

Context: 
Oxidants and Reductants Compounds that are capable of accepting electrons, such as O 2 or F2, are calledoxidants (or oxidizing agents) because they can oxidize other compounds. In the process of accepting electrons, an oxidant is reduced. Compounds that are capable of donating electrons, such as sodium metal or cyclohexane (C6H12), are calledreductants (or reducing agents) because they can cause the reduction of another compound. In the process of donating electrons, a reductant is oxidized. These relationships are summarized in Equation 3.30: Equation 3.30 Saylor URL: http://www. saylor. org/books.
Generated QA: 
<pad> Compounds that are capable of accepting electrons, such as o 2 or f2, are called what?<sep> oxidants</s>
--------------------------------------------------
Context: 
But transgenic animals just have one novel gene. What about an animal with a whole new genome? Could a clone , a genetically exact copy of an organism, be developed using techniques associated wit